In [2]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from pydub import AudioSegment
from  tqdm import tqdm


/home/mmaher/MATCH/MATCH_venv/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# ar BERT read

In [3]:
# Read data & split to text & paths list
p_ar = "/home/mmaher/ASR_DATA/0.ar_data/ALL_DATA/ALL_path_text.txt"

with open(p_ar, encoding="utf8") as f:
    ar_paths_texts = f.readlines()

ar_texts = [(" ".join(a.split(" ")[1:])).replace("\n","").strip() for a in ar_paths_texts]
ar_paths = [ a.split(" ")[0].replace("\n","").strip() for a in ar_paths_texts]


# vectorize
encoded_txt_ar_trans = np.load("/home/mmaher/ASR_DATA/2.BERT/data_ar/BERT_vectors/translated_en_bert.npy")
print("encoded_txt.shape : " , encoded_txt_ar_trans.shape)

encoded_txt.shape :  (529607, 768)


# en BERT data

In [111]:
with open("/home/mmaher/ASR_DATA/2.BERT/data_en/en_data/text.txt", encoding="utf8") as f:
    en_paths_texts = f.readlines()


en_texts = [(" ".join(a.split(" ")[1:])).replace("\n","").strip() for a in en_paths_texts]
en_paths = [ a.split(" ")[0].replace("\n","").strip() for a in en_paths_texts]


In [112]:
from sentence_transformers import SentenceTransformer
model_bert = SentenceTransformer('bert-base-nli-max-tokens')
en_encoded_txt = model_bert.encode(en_texts)
print("encoded_txt.shape : " , en_encoded_txt.shape)

encoded_txt.shape :  (3843, 768)


# Cosine sim

In [79]:
text_ar_en = [] ; sims_en_all_ars = []

for e in tqdm(range(len(en_encoded_txt))):
    try:
        eng_text_enc = en_encoded_txt[e]
        eng_text = en_texts[e]
        eng_path = en_paths[e]

        sims_en_ars = cosine_similarity(eng_text_enc.reshape(1,eng_text_enc.shape[0]),encoded_txt_ar_trans)
        sims_en_all_ars.append(list(sims_en_ars[0]))

        most_sim_ar_index = sims_en_ars.argmax()
        most_sim_ar_text = ar_texts[most_sim_ar_index]
        most_sim_ar_path = ar_paths[most_sim_ar_index]

        t = most_sim_ar_text + "  .@@.  "   + eng_text
        text_ar_en.append(t)

        sound_ar = AudioSegment.from_wav(most_sim_ar_path+".wav")
        sound_en = AudioSegment.from_wav(eng_path)

        combined_sounds = sound_ar + sound_en
        combined_sounds.export("/home/mmaher/ASR_DATA/2.BERT/data_BERT_out/wavs/" + str(e+1) + ".wav", format="wav")

    except:
        print(e)

  9%|▉         | 337/3843 [12:13<2:23:18,  2.45s/it]

336


 19%|█▊        | 712/3843 [26:51<1:57:17,  2.25s/it]

711


 30%|██▉       | 1143/3843 [43:05<1:42:02,  2.27s/it]

...

 52%|█████▏    | 1983/3843 [1:15:47<1:09:05,  2.23s/it]

1982


 81%|████████  | 3096/3843 [1:58:44<25:29,  2.05s/it]  

3095


100%|██████████| 3843/3843 [2:28:24<00:00,  2.32s/it]


In [89]:
s = np.load("/home/mmaher/ASR_DATA/2.BERT/data_BERT_out/sims.npy")

In [90]:
s.shape

(3843, 529607)

In [80]:
with open('/home/mmaher/ASR_DATA/2.BERT/data_BERT_out/text.txt', 'w') as f:
    for line in text_ar_en:
        f.write(line)
        f.write('\n')
        

np.save( "/home/mmaher/ASR_DATA/2.BERT/data_BERT_out/sims.npy", np.asarray(sims_en_all_ars))